In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages
Get:12 https://developer.downlo

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-04-24 01:36:30--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.54MB/s    in 0.2s    

2021-04-24 01:36:31 (5.54 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [41]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Zillow Housing Prices").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [42]:
# Read in data from S3 Bucket
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-netflix/unit-7/zillow_housing_prices.csv"

In [43]:
#Create a temp view of your dataframe
spark.sparkContext.addFile(url)
zillow_housing_data = spark.read.option('header', 'true').csv(SparkFiles.get("zillow_housing_prices.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")
zillow_housing_data.createOrReplaceTempView("zillow")

In [44]:
# Cast 2020-01 column as float and display the average price per state using Spark SQL
spark.sql('''select StateName, sum(cast(`2020-01` as float))/count(`2020-01`) as averagePrice from zillow group by StateName''').show()


+--------------------+------------------+
|           StateName|      averagePrice|
+--------------------+------------------+
|                Utah|          345200.0|
|              Hawaii|         572868.75|
|           Minnesota|256823.59550561797|
|                Ohio| 179073.9393939394|
|              Oregon| 342466.6666666667|
|            Arkansas|162611.42857142858|
|               Texas|266331.35135135136|
|        North Dakota|230916.66666666666|
|        Pennsylvania|246516.66666666666|
|         Connecticut|291745.28301886795|
|            Nebraska|191727.27272727274|
|              Nevada|          327370.0|
|          Washington|438228.30188679247|
|            Illinois| 234485.7142857143|
|            Oklahoma|162641.93548387097|
|District of Columbia|          580100.0|
|            Delaware| 276073.6842105263|
|          New Mexico|          215550.0|
|       West Virginia|          179200.0|
|            Missouri|206980.26315789475|
+--------------------+------------

In [45]:
# Repeat the same cast and average price for the month of February or 2020-02 using SparkSQL
spark.sql('''select StateName, sum(cast(`2020-02` as float))/count(`2020-02`) as averagePrice from zillow group by StateName''').show()


+--------------------+------------------+
|           StateName|      averagePrice|
+--------------------+------------------+
|                Utah|  347562.962962963|
|              Hawaii|          592525.0|
|           Minnesota|261769.76744186046|
|                Ohio|182818.30065359478|
|              Oregon|348658.82352941175|
|            Arkansas|165128.57142857142|
|               Texas| 269829.6703296703|
|        North Dakota|          232660.0|
|        Pennsylvania| 250119.6261682243|
|         Connecticut| 297518.8679245283|
|            Nebraska|          198640.0|
|              Nevada|          327770.0|
|          Washington| 439433.3333333333|
|            Illinois|232948.19277108434|
|            Oklahoma|165754.83870967742|
|District of Columbia|          575700.0|
|            Delaware| 285189.4736842105|
|          New Mexico|          228962.5|
|       West Virginia|          184400.0|
|            Missouri|209020.83333333334|
+--------------------+------------

In [46]:
# Repeat the same cast and average price per state for the month of February, but filter out any states with greater than $200000 average price
less_than_200000 = spark.sql('''with February as 
                  (select StateName, 
                  sum(cast(`2020-02` as float))/count(`2020-02`) as averagePrice 
                  from zillow group by StateName)
                  select * from February where averagePrice <= 200000''')
less_than_200000.show()

+-------------+------------------+
|    StateName|      averagePrice|
+-------------+------------------+
|         Ohio|182818.30065359478|
|     Arkansas|165128.57142857142|
|     Nebraska|          198640.0|
|     Oklahoma|165754.83870967742|
|West Virginia|          184400.0|
|         Iowa|        184290.625|
|     Kentucky|          184020.0|
|    Louisiana|196130.30303030304|
|  Mississippi|         180293.75|
|      Indiana|192643.83561643836|
+-------------+------------------+



In [47]:
# Convert your SparkSQL dataframe into a Pandas dataframe and save your Pandas dataframe into a CSV file
pandas_under_200000 = less_than_200000.toPandas()
pandas_under_200000
pandas_under_200000.to_csv("under_200k.csv")